# Task interface

The task interface to the cluster presents the engines as a fault-tolerant, dynamic load balancing of Workers. In contrast to the direct interface, the task interface does not have direct access to individual engines. As the IPython scheduler assigns the workers, the interface becomes simpler and more powerful at the same time.

The best part, however, is that both interfaces can be used at the same time to leverage their respective strengths. If calculations do not depend on previous results, the task interface is ideal:

## Create an `LoadBalancedView` instance

In [1]:
import ipyparallel as ipp

In [2]:
rc = ipp.Client()

In [3]:
rc = ipp.Client(url_file='/Users/veit/.ipython/profile_mpi/security/ipcontroller-client.json')

In [4]:
rc = ipp.Client(profile='mpi')

In [5]:
lview = rc.load_balanced_view()

`load_balanced_view` is the default view.

<div class="alert alert-block alert-info">

**See also:**

* [Views](https://ipyparallel.readthedocs.io/en/latest/reference/details.html#views)
</div>

## Fast and easy parallelism

### `map()`-LoadBalancedView

In [6]:
lview.block = True
serial_result = map(lambda x:x**10, range(32))
parallel_result = lview.map(lambda x:x**10, range(32))
serial_result==parallel_result

True

### `@lview.parallel()` decorator

In [7]:
@lview.parallel()
def f(x):
    return 10.0*x**4

f.map(range(32))

[0.0,10.0,160.0,…]

## Dependencies

<div class="alert alert-block alert-info">

**Note:**

Please note that the pure ZeroMQ scheduler does not support any dependencies.
</div>

### Function dependencies

`UnmetDependency`

#### `@ipp.require` decorator

#### `@ipp.depend` decorator

#### `dependent` object



### `Dependency`

In [ ]:
client.block=False

ar = lview.apply(f, args, kwargs)
ar2 = lview.apply(f2)

with lview.temp_flags(after=[ar,ar2]):
    ar3 = lview.apply(f3)

with lview.temp_flags(follow=[ar], timeout=2.5)
    ar4 = lview.apply(f3)

> **See also:** Some parallel workloads can be described as [Directed acyclic graph](https://en.wikipedia.org/wiki/Directed_acyclic_graph) (DAG). In [DAG Dependencies](https://ipyparallel.readthedocs.io/en/latest/reference/dag_dependencies.html) we describe using an example how [NetworkX](https://pyviz-tutorial.readthedocs.io/de/latest/matplotlib/networkx.html) is used to represent the task dependencies as DAG.

### `ImpossibleDependency`

`retries` and `resubmit`

## Schedulers

In [ ]:
ipcontroller --scheme=lru

| Scheme        | Description  |
| ------------- | ------------ |
| `lru`         | ***Least Recently Used***: Always assigns the workers to the last used engine. Similar to *round robin*, however, it does not take into account the runtime of each individual task. |
| `plainrandom` | ***Plain Random***: Randomly selects the engine to be run. |
| `twobin`      | ***Two-Bin Random***: Requires `numpy`. Randomly select two engines and use `lru`. This is often better than the purely random distribution, but requires more computational effort. |
| `leastload`   | ***Least Load***: Standard scheme that the engine always assigns tasks with the fewest outstanding tasks. |
| `weighted`    | ***Weighted Two-Bin Random***: Weighted ***Two-Bin Random*** scheme.